<a href="https://colab.research.google.com/github/kookeej/DILAB/blob/main/7.1-7.7/Lab06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab06 - Softmax Classification
===


* 먼저 필요한 모듈들을 import해준다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)        # 항상 똑같은 결과를 보장하기 위해서 사용

# 1. Softmax    
![](https://render.githubusercontent.com/render/math?math=P%28class%3Di%29%20%3D%20%5Cfrac%7Be%5Ei%7D%7B%5Csum%20e%5Ei%7D&mode=display)

* ```softmax```를 이용하여 숫자를 확률로 바꾼다. ```max```값을 부드럽게 뽑아준다는 의미다. 즉, 모든 확률의 합이 1이 되도록 만들어 주는 함수이다.

In [3]:
z = torch.FloatTensor([1, 2, 3])
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


* 모든 hypothesis의 합을 구해보면 1이 나온다.

In [4]:
hypothesis.sum()

tensor(1.)

# 2. Cross Entropy Loss
![](https://render.githubusercontent.com/render/math?math=L%20%3D%20%5Cfrac%7B1%7D%7BN%7D%20%5Csum%20-%20y%20%5Clog%28%5Chat%7By%7D%29&mode=display)

In [5]:
z = torch.rand(3, 5, requires_grad=True)        # |z| = (3, 5)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


* prediction값이 나왔다.

In [6]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


* 랜덤하게 정답을 생성한 결과이다. 정답은 index가 2, 0, 2가 정답이므로, 0.1878, 0.2182, 0.1960이 정답이다.
* class개수는 5개, sample 수는 3개,

이제 one-hot vector의 형식으로 나타낼 수 있다.

In [7]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

 * one-hot vector를 만들어봤다. 이렇게 [2, 0, 2] 부분만 1로 활성화가 되어있다.
 * scatter_ 이므로 ```in-place```연산이다. 바로 값이 교체되어 들어갔다. 즉, dim=1에 대해서 y.unsqueeze(1)에 1을 넣는다.
 * |y| = (3,). |y.unsqueeze(1)| = (3, 1)

In [8]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


```F.softmax()```와 ```F.log_softmax()```를 사용할 수도 있다.

In [9]:
# Low level
torch.log(F.softmax(z, dim=1))

(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [10]:
# High level
F.log_softmax(z, dim=1)

F.nll_loss(F.log_softmax(z, dim=1), y)      # Nagative Log Liklihood

tensor(1.4689, grad_fn=<NllLossBackward>)

또한 pytorch는 ```F.log_softmax()```와 ```F.nll_loss()```를 합친 ```F.cross_entropy```를 사용할 수 있다. 따라서 한번에 계산을 할 수 있다.

In [11]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

필요에 따라 원하는 연산 방법을 선택하면 된다.

# 3. Training with Low-level Cross Entropy Loss

In [12]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)         # 이산확률 분포로 바꿔줘야하므로 정수로 바꿔준다.

In [13]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


* number of samples = m
* number of classes = 3
* dim = 4
* 따라서 4->3으로 가는 Linear regression을 만들었다.

1000번의 epoch를 시도했을 때, cost의 값이 제대로 떨어지는 것을 확인할 수 있다.     
좀 더 쉽게 구현해보자!

In [14]:
  W = torch.zeros((4, 3), requires_grad=True)
  b = torch.zeros(1, requires_grad=True)

  optimizer = optim.SGD([W, b], lr=0.1)

  nb_epochs = 1000
  for epoch in range(nb_epochs + 1):
      z = x_train.matmul(W) + b
      cost = F.cross_entropy(z, y_train)    # one_hot vector를 만들어주는 과정 생략. scatter 안씀.

      optimizer.zero_grad()
      cost.backward()
      optimizer.step()

      if epoch % 100 == 0:
          print('Epoch {:4d}/{} Cost: {:.6f}'.format(
              epoch, nb_epochs, cost.item()
          ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


cost값이 제대로 감소하는 것을 확인할 수 있다.

# 4. High-level Implementation with nn.Module
좀 더 실전에 가깝게 구현해보자.

In [15]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)

    def forward(self, x):                       # |x| = (m, 4) => (m, 3)
        return self.linear(x)

In [16]:
model = SoftmaxClassifierModel()

In [17]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)

    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269521
Epoch 1000/1000 Cost: 0.241922


딥러닝에서 classification을 할 때, 
* binary classification인 경우에는 logistic regression에서 사용한 cost function을 사용하는 것이 맞다. (binary cross entropy(BCE), sigmoid). 
* 여러 개의 class가 존재한다면, cross_entropy, softmax를 쓴다.